In [1]:
import nuclio 
import mlrun

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [3]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [4]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [5]:
# nuclio: start-code

In [6]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import os

In [7]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    # Reading the data from feature service
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        context.logger.info(service.get([{"symbol": key}],as_list = True))
        data = service.get([{"symbol": key}])[0]
        data["symbol"] = key
        stock_list.append(data)
    
    data = [list(item.values()) for item in stock_list]
    
    columns = [{'text': key, 'type': 'object'} for key in stock_list[0].keys()]
    
    #data = [list(item.values()) for item in context.data]
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [8]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-test-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment",
                "news.last_reaction"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [9]:
# nuclio: end-code

In [10]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-03-24 09:00:29,010 [info] Initalizing context & constructing featrure vector
Python> 2021-03-24 09:00:29,054 [info] Reading features from feature vector
Python> 2021-03-24 09:00:29,115 [info] GOOGL
Python> 2021-03-24 09:00:29,130 [info] [[inf, -inf, inf, -inf, 2041.31005859375, 23487.0, '2021-03-23 15:59:00-04:00', 1.6666666666666667, 1.6666666666666667, 1.666666666666667, '2021-03-24 07:15:27']]
Python> 2021-03-24 09:00:29,134 [info] MSFT
Python> 2021-03-24 09:00:29,138 [info] [[inf, -inf, inf, -inf, 237.5, 555171.0, '2021-03-23 15:59:00-04:00', 0.0, 0.0, 0.0, '2021-03-24 08:30:45']]
Python> 2021-03-24 09:00:29,142 [info] AMZN
Python> 2021-03-24 09:00:29,146 [info] [[inf, -inf, inf, -inf, 3137.5, 37334.0, '2021-03-23 15:59:00-04:00', -2.0, -2.0, -2.0, '2021-03-24 08:30:45']]
Python> 2021-03-24 09:00:29,150 [info] AAPL
Python> 2021-03-24 09:00:29,154 [info] [[inf, -inf, inf, -inf, 122.5350036621094, 861260.0, '2021-03-23 15:59:00-04:00', -0.6666666666666666, -0.66666666666

In [16]:
# test remote deployment
from mlrun import code_to_function
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : 'stocks-test-dani',
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [12]:
addr = fn.deploy(project='stocks-test-dani')

> 2021-03-24 09:00:45,632 [info] Starting remote function deploy
2021-03-24 09:00:45  (info) Deploying function
2021-03-24 09:00:45  (info) Building
2021-03-24 09:00:45  (info) Staging files and preparing base images
2021-03-24 09:00:45  (info) Building processor image
2021-03-24 09:00:47  (info) Build complete
2021-03-24 09:00:53  (info) Function deploy complete
> 2021-03-24 09:00:53,223 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:30499


In [14]:
!curl {addr}

[{"columns": [{"text": "sentiments_min_1h", "type": "object"}, {"text": "sentiments_max_1h", "type": "object"}, {"text": "prices_min_1h", "type": "object"}, {"text": "prices_max_1h", "type": "object"}, {"text": "volumes_min_1h", "type": "object"}, {"text": "volumes_max_1h", "type": "object"}, {"text": "price", "type": "object"}, {"text": "volume", "type": "object"}, {"text": "last_updated", "type": "object"}, {"text": "sentiment", "type": "object"}, {"text": "last_reaction", "type": "object"}, {"text": "symbol", "type": "object"}], "rows": [[1.6666666666666667, 1.6666666666666667, Infinity, -Infinity, Infinity, -Infinity, 2041.31005859375, 23487.0, "2021-03-23 15:59:00-04:00", 1.666666666666667, "2021-03-24 07:15:27", "GOOGL"], [0.0, 0.0, Infinity, -Infinity, Infinity, -Infinity, 237.5, 555171.0, "2021-03-23 15:59:00-04:00", 0.0, "2021-03-24 08:30:45", "MSFT"], [-2.0, -2.0, Infinity, -Infinity, Infinity, -Infinity, 3137.5, 37334.0, "2021-03-23 15:59:00-04:00", -2.0, "2021-03-24 08:30:4